# Textbook Data Retrieval 

This notebook provides code to create the data sets used in this textbook. It also contains a summary of the data, their original source location, the license for the data, the date the data were accessed to generate the committed repository version of the original/processed data, and any other relevant meta-information.

Running all cells in this notebook will:

1. obtain all data from their original sources
- process the raw data to create the clean copy
- check that the cleaned data matches with a stored hash
  - if it does not, a warning is raised to the user to possibly update the data, and the new data are stored in a temporary file for inspection
- check that the stored data in the `data/` folder matches the stored hash
  - if it does not, a warning is raised to check the stored data
  
In order to add a new dataset to this notebook, just create a new section in the same format as each section below (make sure it comes *before* the `Validate All Datasets` code block). You need to implement a `retrieve_[your_data_name]` function that builds a string representation of the dataset, store its hash using the `hash_data()` function. The `Validate All Datasets` code block at the end is responsible for looping over all the data and checking it.

## List of Datasets

| **Name** | **Chapters**| **R Dataset** | **Remote Database** |
| -------- | ----------- | ----------- | ---------------- |
| US 2016 Census/Vote Data | 1, 2, 3 | N | N |
| Canadian Movies | 2 | N | Y |
| Historical US Vote | 3 | N | N |
| mtcars | 3 | Y| N |
| Mauna Loa CO2 | 4 | N | N |
| Islands | 4 | Y | N |
| Old Faithful | 4 | Y | N |
| Speed of Light | 4 | Y | N |
| Wisconsin Breast Cancer | 6, 7 | N | N |
| Sacramento Real Estate | 8, 9 | Y | N | 
| Marketing Data | 10 | N | N |



## Load Packages

This notebook uses the following Python3 packages to obtain and process data.

In [1]:
import numpy as np                #for manipulating arrays
import pandas as pd               #for loading/writing/manipulating tabular data
import requests, ftplib           #for downloading files
import os                         #for handling files
import hashlib                    #for validating files
import io                         #for creating byte streams for xlsx files
import rpy2.robjects as robjects  #for obtaining datasets included in R
robjects.r('library(caret)')      #for the Sacramento dataset in R
from bs4 import BeautifulSoup     #for scraping html


R[write to console]: Loading required package: lattice

R[write to console]: Loading required package: ggplot2



## Common Functions

In [2]:
datasets = {}

# This function takes in a string and outputs its SHA1 hash
def hash_data(data):
    data_bytes = data.encode()
    sha1 = hashlib.sha1()
    sha1.update(data_bytes)
    return sha1.hexdigest()

def download_ftp(url, folder_path, filename):
    print('Downloading ' + filename + ' from ' + url)
    raw_data = ''
    try:
        with ftplib.FTP(url) as ftp:
            ftp.login()
            ftp.cwd(folder_path)
            resp = []
            ftp.retrlines('RETR '+filename, callback = lambda ln : resp.append(ln))
            raw_data = '\n'.join(resp)
    except Exception as e:
        print('Exception while downloading ' + filename + ' from ' + url)
        print(e)
    
    return raw_data

def download_http(url):
    print('Downloading from ' + url)
    return requests.get(url).content.decode('utf-8')    

def retrieve_r_table(name):
    print('Obtaining ' + name + ' from the R kernel')
    robjects.r('data('+name+')')
    table = robjects.r(name)
    colnames = list(table.names)
    cols = []
    for colname in colnames:
        if type(table.rx2(colname)) == robjects.vectors.FactorVector:
            levels = list(table.rx2(colname).levels)
            cols.append([levels[lv-1] for lv in list(table.rx2(colname))])
        else:
            cols.append(list(table.rx2(colname)))
    data = ','.join(colnames)+'\n'
    for r in range(len(cols[0])):
        data += ','.join([str(col[r]) for col in cols]) + '\n'
    return data

# US 2016 Census / Vote Data

## Meta-info

- **Source:** [DataUSA](https://datausa.io) and [Federal Election Commission](https://www.fec.gov)
- **Data URL:**
    - Census data: http://datausa.io/api/data?drilldowns=State&measure=Average%20Commute%20Time,Property%20Value,Median%20Household%20Income,Population&year=2016
    - Election data: https://www.fec.gov/documents/1890/federalelections2016.xlsx
- **Date Accessed:** July 5, 2020
- **License:** 

```
You can copy, download or print content for your own use, and you can also include excerpts from Data USA, databases and multimedia products in your own documents, presentations, blogs, websites and teaching materials, provided that suitable acknowledgment of Data USA as source is given.

All requests for commercial use and translation rights should be submitted to usage@datausa.io.
```
- **Further Instructions:** You must run `Rscript create_state_property_vote_variants.R` in this folder to generate the non-csv versions of the data. These are used in the reading data chapter.

## Processing Code

In [46]:
def retrieve_state_property_vote():
    print('Downloading 2016 census data from http://datausa.io')
    #add a user agent header so that the API doesn't return 403
    useragent = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
    #create the query URL
    state_data_url = "http://datausa.io/api/data?drilldowns=State&measure=Average Commute Time,Property Value,Median Household Income,Population&year=2016"
    state_data = requests.get(state_data_url, headers=useragent).json()['data']
    #for some reason, DataUSA API does not return Median Household Income for Puerto Rico or DC
    #insert it manually based on their website
    for state in state_data:
        if state['State'] == 'Puerto Rico':
            state['Median Household Income'] = 20078
        if state['State'] == 'District of Columbia':
            state['Median Household Income'] = 75506
    #for some reason, DataUSA API
    #convert to a list of lists, removing DC, Puerto Rico
    #format numerical entries as either a 2-decimal float or integer
    colnames = ['State', 'Population', 'Property Value', 'Median Household Income', 'Average Commute Time']
    datalines = [[d[colname] if type(d[colname]) == str else format(d[colname], '.2f').rstrip('0').rstrip('.') for colname in colnames] for d in state_data] # if d['State'] != 'Puerto Rico' and d['State'] != 'District of Columbia']
    #rename column names to shorten and avoid spaces
    colnames = ['state', 'pop', 'med_prop_val', 'med_income', 'avg_commute']
    #obtain general presidential election 2016 results data
    print('Downloading 2016 vote data from http://www.fec.gov')
    elec_data_url = "https://www.fec.gov/documents/1890/federalelections2016.xlsx"
    stream = io.BytesIO(requests.get(elec_data_url).content)
    elec_data = pd.read_excel(stream,sheet_name=8)
    #extract one row per state with winner
    elec_data = elec_data[elec_data['WINNER INDICATOR'] == 'W']
    elec_dict = {}
    for i in range(elec_data.shape[0]):        
        party = elec_data.iloc[i]['PARTY']
        #some rows assign the "winner" to combined parties; check the winner name for these rows
        #otherwise convert DEM/REP to long form names
        if party != 'REP' and party != 'DEM': 
            if elec_data.iloc[i]['LAST NAME'] == 'Trump':
                party = 'Republican'
            else:
                party = 'Democratic'
        elif party == 'REP':
            party = 'Republican'
        elif party == 'DEM':
            party = 'Democratic'
        elec_dict[elec_data.iloc[i]['STATE']] = party
    #combine the two data
    data = [','.join(colnames+['party'])]
    for line in datalines:
        data.append(','.join(line)+',' + (elec_dict[line[0]] if line[0] in elec_dict.keys() else "Not Applicable"))
    return '\n'.join(data)

datasets['state_property_vote'] = {}
datasets['state_property_vote']['compare_hash'] = 'f703237fb2da402e9d945de1e76783d014b60cfe'
datasets['state_property_vote']['retrieve_data'] = retrieve_state_property_vote
        

# Historical US Vote Data

## Meta-info

- **Source:** [Wikipedia](https://wikipedia.org)
- **Data URL:** https://en.wikipedia.org/wiki/List_of_United_States_presidential_elections_by_popular_vote_margin
- **Date Accessed:** July 6, 2020
- **License:** None
- **Further Instructions:** Right now, the wrangling chapter uses a few different versions of this data. These are stored in the `data/` folder under the name `historical_vote*.csv` but are not created from the `us_vote` data. These were not made reproducible because the intention is to move away from this US vote data shortly and replace with another dataset.

## Processing Code

In [50]:
def retrieve_us_vote():
    #scrape the wiki HTML
    us_vote_url = 'https://en.wikipedia.org/wiki/List_of_United_States_presidential_elections_by_popular_vote_margin'
    us_vote_html = requests.get(us_vote_url).content
    soup = BeautifulSoup(us_vote_html, 'html.parser')
    tables = soup.find_all('table')
    data = ''
    for table in tables:
        #search for the actual election results table
        if 'wikitable' in table['class']:
            #create the column names synthetically, since the actual HTML table is organized with multiple col name levels
            colnames = ['election_num', 'election_year', 'winner', 'winner_party', 'elec_coll_votes_count',
                        'elec_coll_votes_tot','elec_coll_votes_perc', 'pop_votes_perc', 'pop_votes_perc_marg','pop_votes_count',
                        'pop_votes_count_marg','runnerup','runnerup_party','turnout']
            data += ','.join(colnames)+'\n'
            #obtain the rows of the table
            list_rows = []
            table_rows = table.find_all('tr')   
            for i in range(3, len(table_rows)):
                #convert the entries to just plain text, remove tags
                entries = table_rows[i].find_all('td')
                entries_txt = [entry.text.strip() for entry in entries]
                #names are in the format Lastname,Firstname Lastname. Remove the redundant lastname
                entries_txt[2] = entries_txt[2].split(',')[1]
                entries_txt[10] = entries_txt[10].split(',')[1]
                #if this is the first election, the year is 1788-89. just replace the year with 1788
                if '-' in entries_txt[1]:
                    entries_txt[1] = entries_txt[1].split('-')[0]
                #there is one D.-R. party entry with [Note 1] -- remove that
                if entries_txt[11] == 'D.-R.[Note 1]':
                    entries_txt[11] = 'D.-R.'
                #remove commas and percent symbols
                for j in range(len(entries_txt)):
                    entries_txt[j] = entries_txt[j].replace(',','').replace('%','')
                #split the electoral college vote/total into two entries
                elecvotes = entries_txt[4].split('/')
                entries_txt[4] = elecvotes[0]
                entries_txt.insert(5, elecvotes[1])
                #add this row to the list
                list_rows.append(entries_txt)
            #sort the rows by first element
            list_rows = sorted(list_rows, key = lambda x : int(x[0]))
            #append to the data
            for row in list_rows:
                data += ','.join(row)+'\n'
    return data

datasets['us_vote'] = {}
datasets['us_vote']['compare_hash'] = '1f505f91c81684a48dd631663b3ff8777600bf70'
datasets['us_vote']['retrieve_data'] = retrieve_us_vote



# Motor Trend Car Road Tests Data

## Meta-info

- **Source:** Base R dataset
- **Attribution:** Henderson and Velleman (1981), Building multiple regression models interactively. Biometrics, 37, 391–411. From the 1974 Motor Trends magazine. See https://stat.ethz.ch/R-manual/R-devel/library/datasets/html/mtcars.html
- **Date Accessed:** July 4, 2020
- **License:** None



## Processing Code

In [5]:
def retrieve_mtcars():
    return retrieve_r_table('mtcars')

datasets['mtcars'] = {}
datasets['mtcars']['compare_hash'] = '474fd20770e546b99982923697add9c44409feaf'
datasets['mtcars']['retrieve_data'] = retrieve_mtcars

# Mauna Loa CO2 Data

## Meta-info

- **Source:** [National Ocean and Atmospheric Administration (NOAA)](https://www.esrl.noaa.gov/gmd/ccgg/trends/data.html)
- **Data URL:** ftp://aftp.cmdl.noaa.gov/products/trends/co2/co2_weekly_mlo.txt
- **Date Accessed:** July 7, 2020
- **Attribution:** Dr. Pieter Tans, [NOAA/GML](www.esrl.noaa.gov/gmd/ccgg/trends/) and Dr. Ralph Keeling, [Scripps Institution of Oceanography](https://scrippsco2.ucsd.edu/).
- **License:**


```
# --------------------------------------------------------------------
# USE OF NOAA ESRL DATA
# 
# These data are made freely available to the public and the
# scientific community in the belief that their wide dissemination
# will lead to greater understanding and new scientific insights.
# The availability of these data does not constitute publication
# of the data.  NOAA relies on the ethics and integrity of the user to
# ensure that ESRL receives fair credit for their work.  If the data 
# are obtained for potential use in a publication or presentation, 
# ESRL should be informed at the outset of the nature of this work.  
# If the ESRL data are essential to the work, or if an important 
# result or conclusion depends on the ESRL data, co-authorship
# may be appropriate.  This should be discussed at an early stage in
# the work.  Manuscripts using the ESRL data should be sent to ESRL
# for review before they are submitted for publication so we can
# ensure that the quality and limitations of the data are accurately
# represented.
# 
# Contact:   Pieter Tans (303 497 6678; pieter.tans@noaa.gov)
# 
# File Creation:  Sat Jul  4 05:00:25 2020
# 
# RECIPROCITY
# 
# Use of these data implies an agreement to reciprocate.
# Laboratories making similar measurements agree to make their
# own data available to the general public and to the scientific
# community in an equally complete and easily accessible form.
# Modelers are encouraged to make available to the community,
# upon request, their own tools used in the interpretation
# of the ESRL data, namely well documented model code, transport
# fields, and additional information necessary for other
# scientists to repeat the work and to run modified versions.
# Model availability includes collaborative support for new
# users of the models.
# --------------------------------------------------------------------
#  
#  
# See www.esrl.noaa.gov/gmd/ccgg/trends/ for additional details.
#  
# NOTE: DATA FOR THE LAST SEVERAL MONTHS ARE PRELIMINARY, ARE STILL SUBJECT
# TO QUALITY CONTROL PROCEDURES.
# NOTE: The week "1 yr ago" is exactly 365 days ago, and thus does not run from
# Sunday through Saturday. 365 also ignores the possibility of a leap year.
# The week "10 yr ago" is exactly 10*365 days +3 days (for leap years) ago.
```

## Processing Code

In [6]:
def retrieve_mauna_loa():
    data = download_ftp('aftp.cmdl.noaa.gov', 'products/trends/co2/', 'co2_mm_mlo.txt')
    # remove the lines beginning with # (these are for meta information)
    no_meta_info = [s for s in data.split('\n') if s[0] != '#']
    # replace all whitespace with a single space, strip from beginning and end, keep only first 4 cols
    standardized_whitespace = [','.join([num for num in s.strip().split(' ') if len(num)>0][:4]) for s in no_meta_info]
    # stitch together into a string with col names at the head
    clean_data = 'year,month,date_decimal,ppm\n'+'\n'.join(standardized_whitespace)
    return clean_data

datasets['mauna_loa'] = {}
datasets['mauna_loa']['compare_hash'] = '0a4ee4c99c009424d65ebeeef747ffebd4a8557d'
datasets['mauna_loa']['retrieve_data'] = retrieve_mauna_loa


# Island Landmasses Data

## Meta-info

- **Source:** Base R dataset
- **Attribution:** The World Almanac and Book of Facts, 1975, page 406. See https://stat.ethz.ch/R-manual/R-patched/library/datasets/html/islands.html 
- **Date Accessed:** July 4, 2020
- **License:** None


## Processing Code

In [7]:
def retrieve_islands():
    print('Obtaining islands data from the R kernel')
    isl = robjects.r('islands')
    names = list(isl.names)
    vals = list(isl)
    data = 'landmass,size\n'
    data = data + '\n'.join([d[0]+','+str(int(d[1])) for d in zip(names, vals)])
    return data

datasets['islands'] = {}
datasets['islands']['compare_hash'] = 'c127b83a920db02021d378b4f368b66f90f4dc90'
datasets['islands']['retrieve_data'] = retrieve_islands

# Old Faithful Data

## Meta-info

- **Source:** Base R dataset
- **Attribution:** Azzalini, A. and Bowman, A. W. (1990). A look at some data on the Old Faithful geyser. Applied Statistics, 39, 357–365. doi: 10.2307/2347385. See https://stat.ethz.ch/R-manual/R-patched/library/datasets/html/faithful.html
- **Date Accessed:** July 4, 2020
- **License:** None



## Processing Code

In [8]:
def retrieve_faithful():
    return retrieve_r_table('faithful')

datasets['faithful'] = {}
datasets['faithful']['compare_hash'] = '6e838cd7f190cbb365bec6bb98099052e63d252b'
datasets['faithful']['retrieve_data'] = retrieve_faithful

# Michelson Speed of Light Data

## Meta-info

- **Source:** Base R dataset
- **Attribution:** A. A. Michelson (1882) Experimental determination of the velocity of light made at the United States Naval Academy, Annapolis. Astronomic Papers 1 135–8. U.S. Nautical Almanac Office. (See Table 24.) See https://stat.ethz.ch/R-manual/R-patched/library/datasets/html/morley.html
- **Date Accessed:** July 4, 2020
- **License:** None



## Processing Code

In [9]:
def retrieve_michelson():
    return retrieve_r_table('morley')

datasets['michelson'] = {}
datasets['michelson']['compare_hash'] = 'bb0492aa16d01bb5d6f7a0839d266a607b2e9a6e'
datasets['michelson']['retrieve_data'] = retrieve_michelson

# Wisconsin Breast Cancer Data

## Meta-info

- **Source:** [The UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+(Diagnostic))
- **Data URL:** https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wdbc.data
- **Date Accessed:** July 4, 2020
- **Attribution:** Dr. William H. Wolberg, W. Nick Street, and Olvi L. Mangasarian, University of Wisconsin.
- **License:** None


## Processing Code

In [10]:
def retrieve_unscaled_wdbc():
    data = download_http('https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wdbc.data')
    #create list of variable names
    names = ['ID', 'Class', 'Radius', 'Texture', 'Perimeter', 'Area', 'Smoothness', 'Compactness', 'Concavity', 'Concave_Points', 'Symmetry', 'Fractal_Dimension']
    #remove all but the class label (B/M) and first 10 entries (means of each value)
    data_lines = [line.split(',')[:12] for line in data.split('\n')]
    clean_data = ','.join(names) + '\n' + '\n'.join([','.join(line) for line in data_lines])
    return clean_data

datasets['unscaled_wdbc'] = {}
datasets['unscaled_wdbc']['compare_hash'] = '57dd9bb52f8c5e9f1aa831de7ce199e7a97dffda'
datasets['unscaled_wdbc']['retrieve_data'] = retrieve_unscaled_wdbc

def retrieve_wdbc():
    data = retrieve_unscaled_wdbc()
    tbl = pd.read_csv(io.StringIO(data))
    sel = (tbl.columns != 'ID') & (tbl.columns != 'Class')
    tbl.loc[:,sel] = (tbl.loc[:,sel] - tbl.loc[:,sel].mean())/tbl.loc[:,sel].std()
    out = io.StringIO()
    tbl.to_csv(out, index=False)
    return out.getvalue()
    
datasets['wdbc'] = {}
datasets['wdbc']['compare_hash'] = '5529b157e36fec8d5f688ed8c87c0f2c13bb7dcf'
datasets['wdbc']['retrieve_data'] = retrieve_wdbc

# Sacramento Real Estate Data

## Meta-info

- **Source:** The [caret package](http://topepo.github.io/caret/index.html)
- **Attribution:** The Sacramento Bee. See http://topepo.github.io/caret/data-sets.html
- **Date Accessed:** July 5, 2020
- **License:** None


## Processing Code

In [11]:
def retrieve_sacramento():
    return retrieve_r_table('Sacramento')

datasets['sacramento'] = {}
datasets['sacramento']['compare_hash'] = '04bcb4bb63d13570591e937f93411b3d84802b58'
datasets['sacramento']['retrieve_data'] = retrieve_sacramento

# Marketing Data

## Meta-info

- **Source:** Synthetic, created by the authors. Inspired by https://www.segmentationstudyguide.com/using-cluster-analysis-for-market-segmentation/ .
- **Date Accessed:** July 4, 2020
- **License:** None


## Processing Code

In [12]:
marketing_data = '''loyalty,csat,cluster
7,1,1
7.5,1,1
8,2,1
7,2,1
8,3,1
3,2,3
1,3,3
4,4,3
2,4,3
7,6,2
6,6,2
7,7,2
6,7,2
5,7,2
9,8,2
7,8,2
9,9,2
5,9,2
2,3,3
'''

def retrieve_marketing():
    print('Creating marketing data')
    return marketing_data

datasets['marketing'] = {}
datasets['marketing']['compare_hash'] = '9b49d36e53fd474bb7ca04b879a5219f41589c37'
datasets['marketing']['retrieve_data'] = retrieve_marketing

# Validate All Datasets

In [52]:
print('Validating all datasets\n')
for dnm, ddict in datasets.items():
    #Obtain data from source and verify the hash matches
    print('Processing '+ dnm)
    print('Obtaining data from source...')
    new_data = ddict['retrieve_data']()    
    print('Computing hash...')
    dhash = hash_data(new_data)
    if dhash == ddict['compare_hash']:
        print('Hash matches for fresh copy of ' + dnm + ' obtained from source')
    else:
        print('Hash mismatch for fresh copy of ' + dnm +'; source changed.')
        print('Original Hash: ' + ddict['compare_hash'])
        print('New Hash: ' + dhash)
        print('Saving new cleaned data in ' + dnm+'_new.csv')
        print('If you want to use the new data, remove the old data and rename that file to ' + dnm+'.csv')
        print('Then update the hash in this notebook for ' + dnm + ' to ' + dhash)
        f = open(dnm+'_new.csv', 'w')
        f.write(new_data)
        f.close()
    #Obtain data from storage and verify the hash matches
    print('Obtaining data from storage...')
    try:
        f = open(dnm+'.csv', 'r')
        old_data = f.read()
        f.close()  
        print('Computing hash...')
        dhash = hash_data(old_data)
        if dhash == ddict['compare_hash']:
            print('Hash matches for storage copy of ' + dnm)
        else:
            print('Hash mismatch for stored copy of ' + dnm)
            print('Either update the hash in this notebook to the stored file hash: ' + dhash)
            print('Or investigate why the data appears to have changed.')
            print('Saving new data to ' + dnm+'_new.csv for comparison')
            f = open(dnm+'_new.csv', 'w')
            f.write(new_data)
            f.close()
    except:
        print('Cannot load ' + dnm + ' from storage. Check that the file exists.')   
    print('\n')

Validating all datasets

Processing state_property_vote
Obtaining data from source...
Computing hash...
Hash matches for fresh copy of state_property_vote obtained from source
Obtaining data from storage...
Computing hash...
Hash matches for storage copy of state_property_vote


Processing us_vote
Obtaining data from source...
Computing hash...
Hash matches for fresh copy of us_vote obtained from source
Obtaining data from storage...
Computing hash...
Hash matches for storage copy of us_vote


Processing mtcars
Obtaining data from source...
Obtaining mtcars from the R kernel
Computing hash...
Hash matches for fresh copy of mtcars obtained from source
Obtaining data from storage...
Computing hash...
Hash matches for storage copy of mtcars


Processing mauna_loa
Obtaining data from source...
Computing hash...
Hash matches for fresh copy of mauna_loa obtained from source
Obtaining data from storage...
Computing hash...
Hash matches for storage copy of mauna_loa


Processing islands
Obtain